# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from the file `Oders.csv` located in the `data` folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('C:/Users/user/IronHack/Week2/Labs/lab-df-calculation-and-transformation/data/Orders.csv',index_col = 0)
orders

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20
541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60
541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60
541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
aggregates = orders.groupby('InvoiceNo').sum()

In [4]:
aggregates

,year,month,day,hour,Quantity,UnitPrice,CustomerID,amount_spent
InvoiceNo,,,,,,,,
536365,14070,84,21,56,40,27.37,124950,139.12
536366,4020,24,6,16,12,3.70,35700,22.20
536367,24120,144,36,96,83,58.24,156564,278.73
536368,8040,48,12,32,15,19.10,52188,70.05
536369,2010,12,3,8,3,5.95,13047,17.85
...,...,...,...,...,...,...,...,...
581583,4022,24,10,24,76,3.30,27554,124.60
581584,4022,24,10,24,120,2.57,27554,140.64
581585,42231,252,105,252,278,37.78,331884,329.05


In [5]:
orders = orders.merge(aggregates['amount_spent'],'left',left_on = 'InvoiceNo',right_index=True)

In [6]:
orders.rename(columns={'amount_spent_x':'amount_spent','amount_spent_y':'agg_amount'},inplace = True)

In [7]:
orders["Customer Type"] = pd.qcut(orders["agg_amount"], 
                             q=10,
                             labels=["1","2", "3", "4","5","6","7","8","9",'VIP'])

In [26]:
orders[orders['Customer Type']=='VIP']

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,agg_amount,Customer Type
178,536387,79321,2010,12,3,9,chilli lights,192,2010-12-01 09:58:00,3.82,16029,United Kingdom,733.44,3193.92,VIP
179,536387,22780,2010,12,3,9,light garland butterfiles pink,192,2010-12-01 09:58:00,3.37,16029,United Kingdom,647.04,3193.92,VIP
180,536387,22779,2010,12,3,9,wooden owls light garland,192,2010-12-01 09:58:00,3.37,16029,United Kingdom,647.04,3193.92,VIP
181,536387,22466,2010,12,3,9,fairy tale cottage nightlight,432,2010-12-01 09:58:00,1.45,16029,United Kingdom,626.40,3193.92,VIP
182,536387,21731,2010,12,3,9,red toadstool led night light,432,2010-12-01 09:58:00,1.25,16029,United Kingdom,540.00,3193.92,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540357,581476,23358,2011,12,5,8,hot stuff hot water bottle,18,2011-12-09 08:48:00,1.25,12433,Norway,22.50,2618.89,VIP
540358,581476,23293,2011,12,5,8,set of 12 fairy cake baking cases,32,2011-12-09 08:48:00,0.83,12433,Norway,26.56,2618.89,VIP
540359,581476,23243,2011,12,5,8,set of tea coffee sugar tins pantry,12,2011-12-09 08:48:00,4.95,12433,Norway,59.40,2618.89,VIP
540360,581476,23240,2011,12,5,8,set of 4 knick knack tins doily,6,2011-12-09 08:48:00,4.15,12433,Norway,24.90,2618.89,VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [13]:
orders[orders['Customer Type']=='VIP']['Country'].value_counts()

United Kingdom     27337
EIRE                1997
Netherlands         1788
Germany             1636
France              1436
Switzerland         1089
Norway               735
Australia            665
Spain                475
Portugal             383
Cyprus               309
Finland              247
Channel Islands      233
Japan                231
Israel               198
Singapore            190
Sweden               180
Denmark              161
Italy                143
Belgium              110
USA                  101
Iceland               47
Lebanon               45
Greece                32
Austria               14
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [9]:
# your code here